In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/multi_turn_rewrite/chinese/main')

In [0]:
%tensorflow_version 1.x
!pip install texar

In [0]:
import tensorflow as tf
import texar.tf as tx

import numpy as np
import pprint
import logging

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: True


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

In [0]:
def align_pad(li):
  max_len = max([len(sent) for sent in li])
  for sent in li:
    if len(sent) < max_len:
      sent += [0] * (max_len - len(sent))

In [0]:
def data_generator(f_paths, params):
    for f_path in f_paths:
      with open(f_path) as f:
        print('Reading', f_path)
        for line in f:
          line = line.rstrip()
          h1, h2, q, a = line.split('\t')
          char2idx_fn = lambda x: [params['char2idx'].get(c, len(params['char2idx'])) for c in list(x)]
          h1, h2, q, a = char2idx_fn(h1), char2idx_fn(h2), char2idx_fn(q), char2idx_fn(a)
          align_pad([h1, h2])
          a_in = [1] + a
          a_out = a + [2]
          nested = ({'history': [h1, h2], 'query': q}, (a_in, a_out))
          yield nested

In [0]:
def dataset(is_training, params):
  _shapes = ({'history':[None, None], 'query':[None]}, ([None], [None]))
  _types = ({'history':tf.int32, 'query': tf.int32}, (tf.int32, tf.int32))
  _pads = ({'history':0, 'query':0}, (0, 0))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
def clip_grads(loss):
    variables = tf.trainable_variables()
    pprint.pprint(variables)
    grads = tf.gradients(loss, variables)
    clipped_grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    return zip(clipped_grads, variables)


def rnn_cell():
    def cell_fn():
        cell = tf.nn.rnn_cell.LSTMCell(params['hidden_units'],
                                       initializer=tf.orthogonal_initializer())
        return cell
    if params['dec_layers'] > 1:
      cells = []
      for i in range(params['dec_layers']):
        if i == params['dec_layers'] - 1:
          cells.append(cell_fn())
        else:
          cells.append(tf.nn.rnn_cell.ResidualWrapper(cell_fn(), residual_fn=lambda i,o: tf.concat((i,o), -1)))
      return tf.nn.rnn_cell.MultiRNNCell(cells)
    else:
      return cell_fn()

  
def dec_cell(enc_out, enc_seq_len):
    query, history = enc_out
    query_len, history_len = enc_seq_len

    attn1 = tf.contrib.seq2seq.LuongAttention(
        num_units = params['hidden_units'],
        memory = query,
        memory_sequence_length = query_len)
    
    attn2 = tf.contrib.seq2seq.BahdanauAttention(
        num_units = params['hidden_units'],
        memory = history,
        memory_sequence_length = history_len)
    
    return tf.contrib.seq2seq.AttentionWrapper(
        cell = rnn_cell(),
        attention_mechanism = [attn1, attn2],
        attention_layer_size = [params['hidden_units']//2, params['hidden_units']//2])
    

class TiedDense(tf.layers.Layer):
  def __init__(self, tied_embed, out_dim):
    super().__init__()
    self.tied_embed = tied_embed
    self.out_dim = out_dim
  
  def build(self, input_shape):
    self.bias = self.add_weight(name='bias',
                                shape=[self.out_dim],
                                trainable=True)
    super().build(input_shape)
  
  def call(self, inputs):
    x = tf.matmul(inputs, self.tied_embed, transpose_b=True)
    x = tf.nn.bias_add(x, self.bias)
    return x
  
  def compute_output_shape(self, input_shape):
    return input_shape[:-1].concatenate(self.out_dim)

In [0]:
def sampling_proba_fn():
  global_step = tf.train.get_or_create_global_step()
  return 1 - tf.maximum(tf.train.exponential_decay(1., global_step, 1000, .99), .5)

In [0]:
def bilstm_encode(lstm_cell_fw, lstm_cell_bw, x, words_len):
  t = tf.transpose(x, perm=[1, 0, 2])  # time-major
  o_fw, s_fw = lstm_cell_fw(t, dtype=tf.float32, sequence_length=words_len)
  o_bw, s_bw = lstm_cell_bw(t, dtype=tf.float32, sequence_length=words_len)
  enc_out = tf.concat((o_fw, o_bw), axis=-1)
  enc_out = tf.transpose(enc_out, perm=[1, 0, 2])
  enc_state = tf.concat((s_fw.h, s_bw.h), axis=-1)
  return enc_out, enc_state


def teach_forcing(labels, embedding, enc_out, enc_state, words_len, batch_sz, params, mode, output_proj):
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)

  dec_inputs, dec_outputs = labels
  dec_seq_len = tf.count_nonzero(dec_inputs, 1, dtype=tf.int32)
  dec_inputs = tf.nn.embedding_lookup(embedding, dec_inputs)
  dec_inputs = tf.layers.dropout(dec_inputs, params['dropout_rate'], training=is_training)
  cell = dec_cell(enc_out, words_len)
  
  init_state = cell.zero_state(batch_sz, tf.float32).clone(
      cell_state=enc_state)
  
  if is_training:
    helper = tf.contrib.seq2seq.ScheduledEmbeddingTrainingHelper(
        inputs = dec_inputs,
        sequence_length = dec_seq_len,
        embedding = embedding,
        sampling_probability = sampling_proba_fn(),)
  else:
    helper = tf.contrib.seq2seq.TrainingHelper(
        inputs = dec_inputs,
        sequence_length = dec_seq_len,)
  decoder = tf.contrib.seq2seq.BasicDecoder(
      cell = cell,
      helper = helper,
      initial_state = init_state,
      output_layer = output_proj)
  decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
      decoder = decoder,
      maximum_iterations = tf.reduce_max(dec_seq_len))
  
  return decoder_output.rnn_output


def beam_search(embedding, enc_out, enc_state, words_len, batch_sz, params, output_proj):
  enc_out_t = [tf.contrib.seq2seq.tile_batch(e, params['beam_width']) for e in enc_out]
  enc_state_t = tf.contrib.seq2seq.tile_batch(enc_state, params['beam_width'])
  enc_seq_len_t = []
  for l in words_len:
    if l is not None:
      enc_seq_len_t.append(tf.contrib.seq2seq.tile_batch(l, params['beam_width']))
    else:
      enc_seq_len_t.append(l)
  
  cell = dec_cell(enc_out_t, enc_seq_len_t)
  
  init_state = cell.zero_state(batch_sz*params['beam_width'], tf.float32).clone(
      cell_state=enc_state_t)
  
  decoder = tf.contrib.seq2seq.BeamSearchDecoder(
      cell = cell,
      embedding = embedding,
      start_tokens = tf.tile(tf.constant([1], tf.int32), [batch_sz]),
      end_token = 2,
      initial_state = init_state,
      beam_width = params['beam_width'],
      output_layer = output_proj,
      length_penalty_weight = params['length_penalty'],
      coverage_penalty_weight = params['coverage_penalty'],)
  decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
      decoder = decoder,
      maximum_iterations = params['max_len'],)
  
  return decoder_output.predicted_ids[:, :, 0]

In [0]:
def dynamic_memory_update(query, history):
  proj_1 = tf.layers.Dense(params['hidden_units'], params['activation'], name='attn_proj_1')
  proj_2 = tf.layers.Dense(1, use_bias=False, name='attn_proj_2')
  memory_proj = tf.layers.Dense(params['hidden_units'], params['activation'], name='memory_proj')

  memory = query
  for i in range(params['num_hops']):
      episode = gen_episode(memory,
                            query,
                            history,
                            proj_1,
                            proj_2,)
      memory = memory_proj(tf.concat([memory, episode, query], 1))

  lstm_memory = tf.nn.rnn_cell.LSTMStateTuple(c=memory, h=memory)
  if params['dec_layers'] > 1:
    lstm_memory = tuple(params['dec_layers'] * [lstm_memory])
  
  return lstm_memory


def gen_episode(memory, q_vec, fact_vecs, proj_1, proj_2):
  def gen_attn(fact_vec):
      features = [fact_vec * q_vec,
                  fact_vec * memory,
                  tf.abs(fact_vec - q_vec),
                  tf.abs(fact_vec - memory)]
      feature_vec = tf.concat(features, 1)
      attention = proj_1(feature_vec)
      attention = proj_2(attention)
      return tf.squeeze(attention, 1)
  
  attns = tf.map_fn(gen_attn, tf.transpose(fact_vecs, [1,0,2]))
  attns = tf.transpose(attns)                                      # (B, n_fact)
  attns = params['gating_fn'](attns)                               # (batch, n_fact)
  attns = tf.expand_dims(attns, -1)                                # (B, n_fact, 1)
  episode = tf.matmul(fact_vecs, attns, transpose_a=True)
  episode = tf.squeeze(episode, -1)

  return episode

In [0]:
def forward(features, labels, mode):
    history = features['history']
    query = features['query']
    query_valid_len = tf.count_nonzero(query, 1, dtype=tf.int32)
    
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    batch_sz = tf.shape(query)[0]

    num_history = tf.shape(history)[1]
    history_len = tf.shape(history)[2]
    history = tf.reshape(history, (num_history*batch_sz, history_len))
    history_valid_len = tf.count_nonzero(history, 1, dtype=tf.int32)
    
    
    with tf.variable_scope('Embedding'):
        embedding = tf.Variable(np.load('../vocab/char.npy'),
                                dtype=tf.float32,
                                name='fasttext_vectors')
        def embed_fn(x):
          x = tf.nn.embedding_lookup(embedding, x)
          x = tf.layers.dropout(x, params['dropout_rate'], training=is_training)
          return x
        query = embed_fn(query)
        history = embed_fn(history)
    
    
    with tf.variable_scope('Encoder'):
        lstm_cell_fw = tf.contrib.rnn.LSTMBlockFusedCell(params['hidden_units'])
        lstm_cell_bw = tf.contrib.rnn.LSTMBlockFusedCell(params['hidden_units'])
        lstm_cell_bw = tf.contrib.rnn.TimeReversedFusedRNN(lstm_cell_bw)

        query_out, query_state = bilstm_encode(lstm_cell_fw, lstm_cell_bw, query, query_valid_len)
        history_out, history_state = bilstm_encode(lstm_cell_fw, lstm_cell_bw, history, history_valid_len)

        feat_proj = tf.layers.Dense(params['hidden_units'], params['activation'], name='feature_projection')
        def feat_engine(out, state):
          return feat_proj(tf.concat((tf.reduce_max(out, 1), state), -1))
        query_feat = feat_engine(query_out, query_state)
        history_feat = tf.reshape(feat_engine(history_out, history_state), (batch_sz, num_history, params['hidden_units']))


    with tf.variable_scope('Dynamic_Memory'): 
      dynamic_memory = dynamic_memory_update(query_feat, history_feat)
      query_memory = query_out
      history_memory = tf.reshape(history_out, (batch_sz, num_history*history_len, 2*params['hidden_units']))
      static_memory = [query_memory, history_memory]
      memory_len = [query_valid_len, None]


    with tf.variable_scope('Decoder'):
        output_proj = TiedDense(embedding, len(params['char2idx'])+1)
        
        if is_training or (mode == tf.estimator.ModeKeys.EVAL):
            return teach_forcing(labels, embedding, static_memory, dynamic_memory, memory_len, batch_sz, params, mode, output_proj)
        else:
            return beam_search(embedding, static_memory, dynamic_memory, memory_len, batch_sz, params, output_proj)

In [0]:
def model_fn(features, labels, mode, params):
    logits_or_ids = forward(features, labels, mode)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=logits_or_ids)
    
    dec_inputs, dec_outputs = labels
    if (params['label_smoothing'] <= .0) or (mode == tf.estimator.ModeKeys.EVAL):
      loss_op = tf.contrib.seq2seq.sequence_loss(logits = logits_or_ids,
                                                 targets = dec_outputs,
                                                 weights = tf.to_float(tf.sign(dec_outputs)))
    else:
      loss_op = tf.losses.softmax_cross_entropy(onehot_labels = tf.one_hot(dec_outputs, len(params['char2idx'])+1),
                                                logits = logits_or_ids,
                                                weights = tf.to_float(tf.sign(dec_outputs)),
                                                label_smoothing = params['label_smoothing'],)
      
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step=tf.train.get_or_create_global_step()
        
        decay_lr = tf.train.exponential_decay(
            params['lr'], global_step, 1000, .99)
        
        train_op = tf.train.AdamOptimizer(decay_lr).apply_gradients(
            clip_grads(loss_op), global_step=global_step)
        
        hook = tf.train.LoggingTensorHook({'lr': decay_lr,
                                           'sampling_proba': sampling_proba_fn()},
                                          every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[hook],)
      
    if mode == tf.estimator.ModeKeys.EVAL:
      return tf.estimator.EstimatorSpec(mode=mode, loss=loss_op)

In [0]:
def minimal_test(estimator):
  test_str = '不想吃火锅|吃吧带你去成都|没钱去'
  h1, h2, q = test_str.split('|')
  char2idx_fn = lambda x: [params['char2idx'].get(c, len(params['char2idx'])) for c in list(x)]
  h1, h2, q = char2idx_fn(h1), char2idx_fn(h2), char2idx_fn(q)
  align_pad([h1, h2])
  predicted = list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    x = {'history':np.reshape([h1, h2], (1, 2, len(h1))),
         'query':np.reshape(q, (1, len(q)))},
         shuffle = False)))[0]
  predicted = ''.join([params['idx2char'].get(idx, '<unk>') for idx in predicted if (idx != 0 and idx != 2)])
  print('-'*12)
  print('minimal test')
  print('Q:', test_str)
  print('A:', predicted)
  print('-'*12)

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True

In [0]:
params = {
    'model_dir': '../model/baseline_lstm_scheduled',
    'log_path': '../log/baseline_lstm_scheduled.txt',
    'train_path': ['../data/train_pos.txt', '../data/train_neg.txt'],
    'test_path': ['../data/test_pos.txt',],
    'vocab_path': '../vocab/char.txt',
    'max_len': 30,
    'activation': tf.nn.elu,
    'dropout_rate': .2,
    'hidden_units': 300,
    'dec_layers': 1,
    'num_hops': 3,
    'gating_fn': tf.sigmoid,
    'beam_width': 5,
    'length_penalty': .0,
    'coverage_penalty': .0,
    'lr': 4e-4,
    'clip_norm': .1,
    'label_smoothing': .1,
    'buffer_size': 40000,
    'batch_size': 32,
    'num_patience': 5,
}

In [0]:
params['char2idx'] = get_vocab(params['vocab_path'])
params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

In [0]:
# Create directory if not exist
Path(os.path.dirname(params['log_path'])).mkdir(exist_ok=True)
Path(params['model_dir']).mkdir(exist_ok=True, parents=True)

# Logging
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
fh = logging.FileHandler(params['log_path'])
logger.addHandler(fh)

# Create an estimator
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  model_dir=params['model_dir'],
  config=tf.estimator.RunConfig(
    save_checkpoints_steps=params['buffer_size']//params['batch_size'], keep_checkpoint_max=1),
  params=params)

best_bleu = 0.
history_bleu = []
tf.enable_eager_execution()

while True:
  estimator.train(input_fn=lambda: dataset(is_training=True, params=params))

  minimal_test(estimator)

  loss = estimator.evaluate(input_fn=lambda: dataset(is_training=False, params=params))['loss']
  ppl = np.exp(loss)
  logger.info("Perplexity: {:.3f}".format(ppl))

  # BLEU
  labels = [label for _, (_, label) in dataset(is_training=False, params=params)]
  labels = [j for i in labels for j in i.numpy()]
  labels = [[params['idx2char'].get(idx, '<unk>') for idx in arr if (idx!=0 and idx!=2)] for arr in labels]

  preds = list(estimator.predict(input_fn=lambda: dataset(is_training=False, params=params)))
  assert len(labels) == len(preds)
  preds = [[params['idx2char'].get(idx, '<unk>') for idx in arr if (idx!=0 and idx!=2)] for arr in preds]

  em = [np.all(p == l) for p, l in zip(preds, labels)]
  em = np.asarray(em).mean()

  bleu, bleu_1, bleu_2, bleu_3, bleu_4 = tx.evals.corpus_bleu_moses(list_of_references=[[l] for l in labels], hypotheses=preds, return_all=True)
  logger.info("BLEU: {:.3f}, BELU-1: {:.3f}, BLEU-2: {:.3f}, BLEU-4: {:.3f}, EM: {:.3f}".format(bleu, bleu_1, bleu_2, bleu_4, em))
  history_bleu.append(bleu)

  if bleu > best_bleu:
    best_bleu = bleu
  logger.info("Best BLEU: {:.3f}".format(best_bleu))

  if len(history_bleu) > params['num_patience'] and is_descending(history_bleu):
    logger.info("BLEU not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

INFO:tensorflow:Using config: {'_model_dir': '../model/baseline_lstm_scheduled', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1250, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff4a08e2550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are init

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ../model/baseline_lstm_scheduled/model.ckpt.
Reading ../data/train_pos.txt
Reading ../data/train_neg.txt
INFO:tensorflow:loss = 8.274241, step = 0
INFO:tensorflow:lr = 0.0004, sampling_proba = 0.0
INFO:tensorflow:global_step/sec: 6.35235
INFO:tensorflow:loss = 5.8977127, step = 100 (15.747 sec)
INFO:tensorflow:lr = 0.00039959815, sampling_proba = 0.0010045767 (15.747 sec)
INFO:tensorflow:global_step/sec: 6.90558
INFO:tensorflow:loss = 5.334702, step = 200 (14.482 sec)
INFO:tensorflow:lr = 0.00039919675, sampling_proba = 0.0020080805 (14.483 sec)
INFO:tensorflow:global_step/sec: 7.39151
INFO:tensorflow:loss = 4.9092283, step = 300 (13.528 sec)
INFO:tensorflow:lr = 0.00039879576, sampling_proba = 0.003010571 (13.527 sec)
INFO:tensorflow:global

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/baseline_lstm_scheduled/model.ckpt-1190
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1190 into ../model/baseline_lstm_scheduled/model.ckpt.
Reading ../data/train_pos.txt
Reading ../data/train_neg.txt
INFO:tensorflow:loss = 2.2959192, step = 1190
INFO:tensorflow:lr = 0.0003952445, sampling_proba = 0.011888683
INFO:tensorflow:global_step/sec: 5.96267
INFO:tensorflow:loss = 2.1517417, step = 1290 (16.773 sec)
INFO:tensorflow:lr = 0.00039484748, sampling_proba = 0.012881279 (16.773 sec)
INFO:tensorflow:global_step/sec: 6.90109
INFO:tensorflow:loss = 2.1541986, step = 1390 (14.495 sec)
INFO:tensorflow:lr = 0.00039445085, sampling_proba = 0.013872862 (14.495 sec)
INFO:tensorflow:

...

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/baseline_lstm_scheduled/model.ckpt-36890
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 36890 into ../model/baseline_lstm_scheduled/model.ckpt.
Reading ../data/train_pos.txt
Reading ../data/train_neg.txt
INFO:tensorflow:loss = 1.2358968, step = 36890
INFO:tensorflow:lr = 0.0002760848, sampling_proba = 0.30978805
INFO:tensorflow:global_step/sec: 6.4178
INFO:tensorflow:loss = 1.2418605, step = 36990 (15.588 sec)
INFO:tensorflow:lr = 0.00027580743, sampling_proba = 0.31048143 (15.588 sec)
INFO:tensorflow:global_step/sec: 6.98854
INFO:tensorflow:loss = 1.2057742, step = 37090 (14.310 sec)
INFO:tensorflow:lr = 0.0002755304, sampling_proba = 0.31117404 (14.310 sec)
INFO:tensorflow:global_step/sec: 7.30065
INFO:tensorflow:loss = 1.2111685, step = 3719

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/baseline_lstm_scheduled/model.ckpt-38080
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 38080 into ../model/baseline_lstm_scheduled/model.ckpt.
Reading ../data/train_pos.txt
Reading ../data/train_neg.txt
INFO:tensorflow:loss = 1.2232927, step = 38080
INFO:tensorflow:lr = 0.00027280248, sampling_proba = 0.31799376


KeyboardInterrupt: ignored